In [ ]:
import math
import pickle
import os
import json
import uuid

import numpy as np
import open3d as o3d
from tqdm import tqdm
from numpy.random import default_rng
from compas.geometry import oriented_bounding_box_numpy
import ifcopenshell.template
import ifcopenshell

In [ ]:
# constants
data_path = "/mnt/f/datasets/AVEVA_newfacility/AVEVA_newfacility/exports/"
save_path = "../cloi/"
shapes = ['Pipes', 'Flanges', 'Elbows']
labels = ['TUBE', 'FLANGE', 'ELBOW', 'TEE', 'BEND']

rng = default_rng()

In [ ]:
def sq_distance(x1, y1, z1, x2, y2, z2):
    return ((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)


def vector_norm(vec):
    den = math.sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)
    return [vec[0]/den, vec[1]/den, vec[2]/den]


# get center, dimensions and direction of cloud
def get_dimensions (element_coords):
    bbox = oriented_bounding_box_numpy(element_coords)
    center = [(bbox[0][i] + bbox[6][i])/2 for i in range(3)]
    
    # identify box orientation
    l1 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[1][0], bbox[1][1], bbox[1][2]))
    l2 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[3][0], bbox[3][1], bbox[3][2]))
    l3 = math.sqrt(sq_distance(bbox[0][0], bbox[0][1], bbox[0][2],
                               bbox[4][0], bbox[4][1], bbox[4][2]))
    lengths = [l1, l2, l3]
    
    dominant_axis = lengths.index(max(lengths))
    if dominant_axis == 0:
        dominant_direction = vector_norm([bbox[0][i] - bbox[1][i] 
                                          for i in range(3)])
    elif dominant_axis == 1:
        dominant_direction = vector_norm([bbox[0][i] - bbox[3][i] 
                                          for i in range(3)])
    else:
        dominant_direction = vector_norm([bbox[0][i] - bbox[4][i] 
                                          for i in range(3)])
        
    return(center, lengths, dominant_direction)




In [ ]:
def rot2eul(R):
    beta = -np.arcsin(R[2,0])
    alpha = np.arctan2(R[2,1]/np.cos(beta),R[2,2]/np.cos(beta))
    gamma = np.arctan2(R[1,0]/np.cos(beta),R[0,0]/np.cos(beta))
    return np.array((alpha, beta, gamma))


def get_labelcloud_bbox(points, label):
    pcl = o3d.geometry.PointCloud()
    pcl.points = o3d.utility.Vector3dVector(points)
    oriented_bounding_box = pcl.get_oriented_bounding_box()

    r = [math.degrees(x) for x in rot2eul(oriented_bounding_box.R)]
    c = oriented_bounding_box.center
    e = oriented_bounding_box.extent

    bbox = {
        'name': label,
        'centroid': {
            'x': c[0],
            'y': c[1],
            'z': c[2]
        },
        'dimensions': {
            'length': e[0],
            'width': e[1],
            'height': e[2]
        },
        'rotations': {
            'x': r[0],
            'y': r[1],
            'z': r[2]
        }
    }
        
    return bbox

In [ ]:
# create smaller cleaned cloud by removing outliers and downsampling
def refine_cloud(file_path, n_points=1000):
    # read points
    f = open(file_path, 'r')
    points = f.readlines()
    points = [p.strip().split(' ')[:3] for p in points]
    for i in range(len(points)):
        points[i] = [float(p) for p in points[i]]
    
    # create cloud
    points = np.array(points)
    #print(points.shape)
    if len(points) == 0:
        return None
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    # outlier removal
    # TODO: experiment with parameters and try radius outlier removal
    voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.005)
    cl, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=20,
                                                        std_ratio=2.0)

    # further down sampling
    if len(ind) > n_points:
        #print('l', len(ind))
        sub_ind = rng.choice(len(ind), size=n_points, replace=False)
        cl = cl.select_by_index(sub_ind)
    
    # o3d.io.write_point_cloud("sync.ply", cl)
    
    return np.asarray(cl.points)

In [ ]:
dataset = {}
for i, shape in enumerate(shapes):
    clouds = []
    elements = os.listdir(data_path + shape)

    points = 0
    for el in tqdm(elements):
        cl = refine_cloud(data_path + shape + '/' + el)
        if cl is not None:
            #print(len(cl))
            clouds.append(cl)
            points += len(cl)
    
    print(len(clouds), points)
    dataset[labels[i]] = clouds


In [ ]:
with open(save_path + 'clouds.pkl', 'wb') as f:
    pickle.dump(dataset, f)

In [ ]:
with open(save_path + 'clouds.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [ ]:
# create and visualize a combined cloud
full = []
for label in dataset:
    merged = np.concatenate(dataset[label])
    full.append(merged)
    print(merged.shape)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(merged)
    o3d.io.write_point_cloud(save_path + label + ".ply", pcd)
    
full = np.concatenate(full)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(full)
o3d.io.write_point_cloud(save_path + "combined.ply", pcd)

In [ ]:
# convert to graph dataset node info, and generate labelcloud annotation

def process_nodes(dataset):
    
    nodes = []
    points = []
    bboxes = []
    error_count = 0
    count = 0
    
    for j in dataset:
        for i, el in tqdm(enumerate(dataset[j])):
            try:
                center, lengths, dominant_direction = get_dimensions(el)
                node = [j, center, lengths, dominant_direction, 
                              count]
                
                bbox = get_labelcloud_bbox(el, j)
                
                count += 1
                bboxes.append(bbox)
                nodes.append(node)
                points.append(el)

            except Exception as e:
                error_count += 1
                print(len(el))
                print(e)

    print(error_count)
    return([nodes, points], bboxes)


In [ ]:
node_info, bboxes = process_nodes(dataset)

In [ ]:
    with open(save_path + 'data.json', 'w') as f:
        json.dump({'objects':bboxes}, f)

In [ ]:
print(len(node_info), len(node_info[0]), len(node_info[1]))

with open(save_path + 'nodes_cloi.pkl', 'wb') as f:
    pickle.dump(node_info, f)

In [ ]:
# TODO convert point dataset to IFC for visualization
output_file = save_path + 'cloi_vis.ifc'
# f2 = ifcopenshell.open() # new empty file
# f2.add(wall)
# f2.write(output_file)

In [ ]:
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)


In [ ]:
f = ifcopenshell.template.create(schema_identifier='IFC4')


In [ ]:
el_id = 10

pointlist = f.createIfcCartesianPointList3D(node_info[1][el_id].tolist())
